In [ ]:
import json
import cv2
import math
import numpy as np
from enum import Enum

from google.colab.patches import cv2_imshow

In [ ]:
def myEnum(*sequential, **named):
    enums = dict(zip(sequential, range(len(sequential))), **named)
    return type('Enum', (), enums)

In [ ]:
def calculate_vector(x1, y1, x2, y2):
    point_a = np.array([x1, y1])
    point_b = np.array([x2, y2])
    vector = point_b - point_a
    return vector

In [ ]:
PI = math.pi
keys = myEnum(
                "nose",
                "head_bottom",
                "head_top",
                "left_ear",
                "right_ear",
                "left_shoulder",
                "right_shoulder",
                "left_elbow",
                "right_elbow",
                "left_wrist",
                "right_wrist",
                "left_hip",
                "right_hip",
                "left_knee",
                "right_knee",
                "left_ankle",
                "right_ankle"
            )

length = [[],
           [ ],
           [ ],
           [],
           [],
           [ ],
           [ ],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
           [],
          []
          ]

In [ ]:
len(length)

In [ ]:
# Standard vector 계산

import os
import json
import numpy as np


standardVector = {}

def calculate_joint_vectors(target,joints):
    # 관절 간의 벡터를 저장할 리스트
    result = []

    # 관절 간의 벡터 계산
    for j in joints:
        joint_vector = calculate_vector(data[0]['keypoints'][target][0],data[0]['keypoints'][target][1]\
                         ,data[0]['keypoints'][j][0],data[0]['keypoints'][j][1])
        result.append(joint_vector)

    return result

# POSES 딕셔너리 초기화
poses = {
    '1_Address': [],
    '2_Takeback': [],
    '3_Backswing': [],
    '4_Topswing': [],
    '5_Downsing': [],
    '6_Impact': [],
    '7_Follow': [],
    '8_Finish': []
}


# Standard 폴더의 이미지 파일 목록 가져오기
standard_files = os.listdir('/content/drive/MyDrive/ytin/Standard2')

# 기준 이미지들을 순회하면서 벡터 계산
for filename in standard_files:
    # JSON 파일 읽기
    with open('/content/drive/MyDrive/ytin/Standard2/' + filename, 'r') as json_file:
        data = json.load(json_file)

    # 계산
    right_shoulder = calculate_joint_vectors(keys.right_shoulder, [keys.left_shoulder, keys.right_elbow, keys.right_hip, keys.right_knee])
    left_shoulder = calculate_joint_vectors(keys.left_shoulder, [keys.right_elbow, keys.left_knee, keys.right_knee, keys.left_elbow])
    left_elbow = calculate_joint_vectors(keys.left_elbow, [keys.right_knee, keys.right_hip, keys.left_knee, keys.left_wrist])
    right_wrist = calculate_joint_vectors(keys.right_wrist, [keys.right_elbow, keys.right_knee, keys.right_hip, keys.left_knee])
    left_wrist = calculate_joint_vectors(keys.left_wrist, [keys.right_knee, keys.right_hip, keys.right_shoulder, keys.left_shoulder])
    right_hip = calculate_joint_vectors(keys.right_hip, [keys.left_hip, keys.right_knee])
    vector = right_shoulder + left_shoulder + left_elbow + right_wrist + right_hip + left_wrist
    # joint_coordinates = np.array(data[0]['keypoints'])

    # 관절 간의 벡터 계산
    # joint_vectors = calculate_joint_vectors(joint_coordinates)

    # 벡터 값을 POSES 딕셔너리에 저장
    pose_name = os.path.splitext(filename)[0]
    standardVector[pose_name] = vector

# 결과 출력

print(standardVector)


In [ ]:
standardVector['1_Address'], len(standardVector['1_Address'])

In [ ]:
import os
import json

address = ''
address_list = []
add_diff = 0

for (path, dir, files) in os.walk("/content/drive/MyDrive/Vector_Nice/pred_Vector_image/"):    #json파일이 있는 경로 모든 폴더 가져옴
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.json':   # json파일 읽어오기
          path = '/content/drive/MyDrive/Vector_Nice/pred_Vector_image/'
          path = path+filename
         # print(path)

          with open(path, "r") as json_file:
            data = json.load(json_file)

# 임의로 지정한 관절 포인트간 벡터 계산
          right_shoulder = calculate_joint_vectors(keys.right_shoulder, [keys.left_shoulder, keys.right_elbow, keys.right_hip, keys.right_knee])
          left_shoulder = calculate_joint_vectors(keys.left_shoulder, [keys.right_elbow, keys.left_knee, keys.right_knee, keys.left_elbow])
          left_elbow = calculate_joint_vectors(keys.left_elbow, [keys.right_knee, keys.right_hip, keys.left_knee, keys.left_wrist])
          right_wrist = calculate_joint_vectors(keys.right_wrist, [keys.right_elbow, keys.right_knee, keys.right_hip, keys.left_knee])
          left_wrist = calculate_joint_vectors(keys.left_wrist, [keys.right_knee, keys.right_hip, keys.right_shoulder, keys.left_shoulder])
          right_hip = calculate_joint_vectors(keys.right_hip, [keys.left_hip, keys.right_knee])
          vector = right_shoulder + left_shoulder + left_elbow + right_wrist + right_hip + left_wrist


# 추정하려는 자세와 벡터의 절대값 차이를 합산
          i = 0
          diff_sum = 0
          while i < len(length):
            diff_sum += np.sum(np.abs(vector[i] - standardVector['6_Impact'][i]))
            i +=1
          print(diff_sum)
#  1_Address   2_Takeback   3_Backswing  4_Topswing  5_Downsing   6_Impact  7_Follow  8_Finish

# 차이값이 가장 작은 이미지를 현재 추정하는 자세로 교체
          if address=='':
            address = filename
            add_diff = diff_sum

          else:
            if diff_sum<add_diff:
              address = filename
              add_diff = diff_sum

print(address)
